In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras import Model
import matplotlib.pyplot as plt
import scipy
import urllib.request
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop

In [4]:
train = ImageDataGenerator(rescale = 1./255.)

test = ImageDataGenerator(rescale = 1./255.)

train_dataset = train.flow_from_directory("data/Train",
                                          target_size=(150,150),
                                          batch_size = 10,
                                          class_mode = 'binary')
                                         
test_dataset = test.flow_from_directory("data/Test",
                                      target_size=(150,150),
                                      batch_size =10,
                                      class_mode = 'binary')

Found 146 images belonging to 2 classes.
Found 36 images belonging to 2 classes.


In [3]:
# inceptionv3 = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
# urllib.request.urlretrieve(
#     inceptionv3, 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
# local_weights_file = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
# pre_trained_model = InceptionV3(input_shape=(150,150,3),include_top=False,weights=None)
# pre_trained_model.load_weights(local_weights_file)
# for layer in pre_trained_model.layers:
#   layer.trainable=False

In [5]:
# last_layer =  pre_trained_model.get_layer('mixed7')
# last_ouput = last_layer.output
# x = keras.layers.Flatten()(last_ouput)
# x = keras.layers.Dense(512, activation='relu')(x)
# x = keras.layers.Dropout(0.2)(x)
# x = keras.layers.Dense(1, activation='sigmoid')(x)

In [6]:
# model = Model(pre_trained_model.input, x)

# model.compile(optimizer=RMSprop(learning_rate=0.0001),
#               loss='binary_crossentropy',
#               metrics=['acc'])

# model.fit(train_dataset,
#     epochs=15,
#     validation_data=test_dataset,
# )

In [7]:
model = keras.Sequential()

# Convolutional layer and maxpool layer 1
model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 2
model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 3
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 4
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Flatten())

# Hidden layer with 512 neurons and Rectified Linear Unit activation function 
model.add(keras.layers.Dense(512,activation='relu'))

# Output layer with single neuron which gives 0 for Anemia or 1 for NonAnemia 
model.add(keras.layers.Dense(1,activation='sigmoid'))

In [8]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [9]:
model.fit(train_dataset,
     epochs = 15,
     validation_data = test_dataset       
)

Epoch 1/15


15/15 [==============================] - 4s 119ms/step - loss: 0.4972 - accuracy: 0.7945 - val_loss: 0.0792 - val_accuracy: 1.0000
Epoch 2/15
15/15 [==============================] - 1s 92ms/step - loss: 0.0375 - accuracy: 1.0000 - val_loss: 0.0308 - val_accuracy: 1.0000
Epoch 3/15
15/15 [==============================] - 1s 86ms/step - loss: 0.0799 - accuracy: 0.9726 - val_loss: 0.0140 - val_accuracy: 1.0000
Epoch 4/15
15/15 [==============================] - 1s 87ms/step - loss: 0.0064 - accuracy: 1.0000 - val_loss: 0.0086 - val_accuracy: 1.0000
Epoch 5/15
15/15 [==============================] - 1s 89ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.0227 - val_accuracy: 0.9722
Epoch 6/15
15/15 [==============================] - 2s 103ms/step - loss: 3.0192e-04 - accuracy: 1.0000 - val_loss: 2.2559e-08 - val_accuracy: 1.0000
Epoch 7/15
15/15 [==============================] - 2s 100ms/step - loss: 1.8266e-07 - accuracy: 1.0000 - val_loss: 1.1225e-06 - val_accuracy:

In [10]:
def predictImage(filename):
    img1 = image.load_img(filename,target_size=(150,150))
    
    plt.imshow(img1)
 
    Y = image.img_to_array(img1)
    
    X = np.expand_dims(Y,axis=0)
    value_predicted = model.predict(X)
    print(value_predicted)
    if value_predicted >= 0.5:
        plt.xlabel("NonAnemia",fontsize=30)
    else:
        plt.xlabel("Anemia",fontsize=30)

In [12]:
# predictImage(r"")

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 17, 17, 128)       0

In [12]:
# model.save("model.h5")